# 출력파서 (`Outputparser`)
- LLM 의 출력을 더 유용/ 구조화된 형태로 변환함.
- 구조화: LLM의 자유 형식 출력을 구조화된 데이터로 변환 
- 일관성: 항상 일관된 출력형식 -> 후속 처리에 용이 
- 유연성: 다양한 출력 형식(JSON, list, dict) 으로 변환 가능 


## PydanticOutputparser
- 이 파서는 Pydantic 모델을 사용하여 LLM의 출력을 특정 스키마(형식)을 가진 객체로 변환한다. 

In [73]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model='gpt-4.1-nano')

In [74]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [75]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """아래 이메일 내용중 중요한 것만 추출해.
    ---
    {email_conversation}
    """
)

# print(prompt.format(email_conversation=email_conversation))

chain =prompt |llm 
print('---출력파서없는 요약--')
print(chain.invoke({'email_conversation':email_conversation}).content)


---출력파서없는 요약--
1. 요청 사항: "ZENESIS" 자전거에 대한 상세 브로슈어(기술 사양, 배터리 성능, 디자인) 요청.
2. 미팅 일정: 다음 주 화요일(1월 15일) 오전 10시에 귀사 사무실에서 미팅 제안.


In [76]:
class EmailSummary(BaseModel):
    person:str = Field(description='메일 보낸 사람')
    email:str = Field(decroption= '보낸사람의 메일주소')
    subject:str = Field(description='메일제목')
    summary: str = Field(description='메일 본문 요약')
    date:str = Field(description='메일에 언급된 미팅 날짜와 시간')

parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions()) #파서 생성  pydanticOutputParser에 정의한 EmailSummary 모델을 전달해서 파서생성 

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"decroption": "보낸사람의 메일주소", "title": "Email", "type": "string"}, "subject": {"description": "메일제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [77]:
prompt =PromptTemplate.from_template(
    """
너는 요약의 신 어시스턴트야. 아래 질문에 맞게 답변을 한국어로 만들어줘 
질문: {question}
EMAIL내용: {email_conversation}
형식: {format}

"""
)
#프롬프트 변수들 중 일부만 채우기 
prompt.partial(format= parser.get_format_instructions()) #프롬프트 수정: get_format_instructions()메서드로 Pydantic 모델에 맞는 출력 형식을 LLM 에게 
# 명시적으로 지시하는 프롬프트 템플릿을 만든다. 

PromptTemplate(input_variables=['email_conversation', 'question'], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "메일 보낸 사람", "title": "Person", "type": "string"}, "email": {"decroption": "보낸사람의 메일주소", "title": "Email", "type": "string"}, "subject": {"description": "메일제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문 요약", "title": "Summary", "type": "string"}, "date": {"description": "메일에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "emai

In [78]:
res = chain.invoke( 
    {
        'question': '이메일 내용중 중요한 내용을 추출해 줘!',
        'email_conversation': email_conversation # 올바른 변수명
    }
)
 # chain 을 통해 LLM이 이메일 내용을 분석하고 EmailSummary 모델에 정의된 필드에 맞춰 구조화된 객체 (res)를 반환한다. 단순텍스트요약보다 유용하게 데이터 활용하게 함 

# CSV Parser
- LLM 출력을 쉼표로 구분된 문자열 리스트로 변환한다. 

In [81]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# CSV 파서의 안내사항 확인
print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template='List 5 {subject}.\n{format_instructions}',
    input_variables=['subject'],
    partial_variables={'format_instructions': output_parser.get_format_instructions()}
)
print('----')
print(prompt.format(subject='중국집대표메뉴'))

chain1 = prompt | llm
csv_res = chain1.invoke({'subject': '맥도날드 대표메뉴'}).content


chain2 = prompt | llm | output_parser
list_res = chain2.invoke({'subject': '국민주식'})


Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
----
List 5 중국집대표메뉴.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


## structured Output Parser
- dict 형식 -> 멍청한 모델에도 적용 가능 

In [ ]:
from langchain.output_parsers import  ResponseSchema, StructuredOutputParser


In [ ]:
response_schema = [
    ResponseSchema(name= 'answer', type='string', description='사용자의 질문에 대한 답변'),
    ResponseSchema(name= 'source', description='질문에 답하기 위해 사용된 출처 웹사이트 주소')
]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [ ]:
print(output_parser.get_format_instructions())

prompt = PromptTemplate(
    template='사용자 질문에 최선을 다해 답변한다. \n{format_instructions}\n{question}',
    input_variables=['question'],
    partial_variables={'format_instructions': output_parser.get_format_instructions}
)
print('---')
print(prompt.format(question="AI란 무엇인가요?"))

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": str  // 사용자의 질문에 대한 답변
	"source": string  // 질문에 답하기 위해 사용된 출처 웹사이트 주소
}
```
---
사용자 질문에 최선을 다해 답변한다. 
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": str  // 사용자의 질문에 대한 답변
	"source": string  // 질문에 답하기 위해 사용된 출처 웹사이트 주소
}
```
AI란 무엇인가요?


In [ ]:
chain = prompt | llm | output_parser

chain.invoke({'question': '처서는 언제야?'})

{'answer': '처서는 4월 1일입니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%B2%BC%EC%84%9C'}

In [ ]:
for token in chain.stream({'question':'처서는 언제야?'}):
    print(token)

# stream: LLM 이 토큰을 하나씩 생성하는 과정을 실시간으로 받아보는 방식 (토큰단위로 조금씩 흘려보내줌  채팅창에 글자 하나씩 입력되는 늮ㅁ )
#parser 는 출력데이터 구조(결과를 구조화하기 


{'answer': '처서는 한국 전통의 명절인 설날(음력 1월 1일)을 의미하며, 이는 매년 음력 날짜에 따라 달라집니다. 자세한 날짜는 음력 달력에 따라 결정됩니다.', 'source': 'https://ko.wikipedia.org/wiki/%EC%B2%98%EC%84%9C'}


- 원하는 DataFrame 을 분석할 수 있음.
- 답변 또한 판다스DF로 반환 

In [ ]:
from langchain.output_parsers import PandasDataFrameOutputParser
import seaborn as sns

titanic_df = sns.load_dataset('titanic')


In [ ]:
parser= PandasDataFrameOutputParser(dataframe=titanic_df)

print(parser.get_format_instructions())

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [ ]:
prompt = PromptTemplate( #모델에 넣을 프롬프트(질문 템플릿 정의)
    template='Answer the user query.\n{format_instructions}\n{query}',
    input_variables=['query'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)
chain = prompt | llm |parser

In [ ]:
res =chain.invoke({'query': 'age 컬럼을 조회해 줘'})

In [ ]:
res = chain.invoke({'query':'age 컬럼의 평균을 구해줘'})

{'age': 0      22.0
 1      38.0
 2      26.0
 3      35.0
 4      35.0
        ... 
 886    27.0
 887    19.0
 888     NaN
 889    26.0
 890    32.0
 Name: age, Length: 891, dtype: float64}

In [ ]:
res =chain.invoke({'query':'fare 컬럼 평균을 구해줘'})

In [ ]:
res['mean']

np.float64(32.204207968574636)

## Datetime Output parser
- 날짜 출력용

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
parser =DatetimeOutputParser(format='%Y-%m-%d')

prompt =PromptTemplate(
    template='사용자 질문에 답해라.\n{format_intructions}\n{question}',
    input_variables=['question'],
    partial_variables={
        'format_instructions':parser.get_format_instructions()
    }
)


In [ ]:
chain = prompt |llm | parser
#datetime 인스턴스로 만들어 줌! 
chain.invoke({'question': '내가 로또를 사서 당첨될 날은?'})

KeyError: "Input to PromptTemplate is missing variables {'email_conversation', 'format'}.  Expected: ['email_conversation', 'format', 'question'] Received: ['question']\nNote: if you intended {email_conversation} to be part of the string and not a variable, please escape it with double curly braces like: '{{email_conversation}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "

## Enum Output Parser(Enumerated Type -> 열거형)
- 정해진 목록중에 답을 고르도록 

In [ ]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser # <--- 이 부분을 추가합니다.
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

llm = OpenAI() # LLM 모델을 사용 예시로 추가

class Colors(Enum):
    RED = '빨간색'
    GREEN = '초록색'
    BLUE = '파란색'
    YELLOW = '노란색'
parser = EnumOutputParser(enum=Colors)

prompt = PromptTemplate(
    template='다음 물체는 어느색깔에 가깝나요?\n{object}\n{instructions}',
    partial_variables={'instructions': parser.get_format_instructions()}
)

chain = prompt | llm | parser

# '병아리'는 Colors 열거형에 정의된 값('빨간색', '초록색', '파란색')과 직접적으로 일치하지 않으므로
# LLM이 가장 적절하다고 판단하는 값을 반환할 것입니다.
result = chain.invoke({'object': '병아리'}) 

print(result) # 결과 출력

## 프롬프트 조정이 중요하다 \n 을 한번 더 치면 병아리 노란색이라고 답장을 해준다. 

Colors.YELLOW
